In [ ]:
import torch
import torch.nn as nn
import numpy as np
from tqdm.auto import tqdm
from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

MAX_LEN = 256

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

In [ ]:
# load huggingface token from file .env
import os
from dotenv import load_dotenv

load_dotenv()

AUTH_TOKEN = os.getenv('HUGGINGFACE_TOKEN')

MRC_PATH = 'nguyenvulebinh/vi-mrc-base'

In [ ]:
TRAIN_PATH = 'data/span_detection_datasets_split_word_IOB/train.jsonl'
DEV_PATH = 'data/span_detection_datasets_split_word_IOB/dev.jsonl'
TEST_PATH = 'data/span_detection_datasets_split_word_IOB/test.jsonl'

In [ ]:
# function read jsonl file as dataframe
import pandas as pd
import json

def read_jsonl_to_dataframe(file_path):
    data = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_obj = json.loads(line)
                data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON: {e}")

    df = pd.DataFrame(data)

    return df

In [ ]:
import json

# load tag_to_id
with open('data/tag_to_id.json', 'r') as f:
    tag_to_id = json.load((f))

# load train and dev data

train_data = read_jsonl_to_dataframe(TRAIN_PATH)
dev_data = read_jsonl_to_dataframe(DEV_PATH)


train_sentences = list(train_data.text.apply(lambda x: " ".join(x)))
dev_sentences = list(dev_data.text.apply(lambda x: " ".join(x)))

train_labels = list(train_data.labels)
dev_labels = list(dev_data.labels)